<a href="https://colab.research.google.com/github/GaneshPechetti/Diarization/blob/main/SVPD_DiarizationPrototype_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Speaker Diarizaton model_1
  Achitecture :    

        * Step 1 : loading libraries 
        * Step 2 : Passing the audio to the silero vad model (if necessary)
        * Step 3 : Preparing the audio file
        * Step 4 : Passing to the hugging face model
        * Step 5 : Creating a json file using the rttm file
        * Step 6 : Generating the audio files for every different speaker which consists of their only their voice

#VAD
In this section we are using silero VAD model to for Voice Activity Detection. 

In [ ]:
!pip install -q torchaudio

SAMPLING_RATE = 16000

import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint

In [ ]:
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime
  
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

In [ ]:
SAMPLING_RATE = 16000
audio_file = "" # set path to the audio file
wav = read_audio(audio_file, sampling_rate=SAMPLING_RATE)
# get speech timestamps from full audio file

speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=SAMPLING_RATE)
pprint(speech_timestamps)

In [ ]:
# merge all speech chunks to one audio
save_audio('only_speech_.wav',
           collect_chunks(speech_timestamps, wav), sampling_rate=SAMPLING_RATE) 

#Diarization

In [ ]:
pip install -qq https://github.com/pyannote/pyannote-audio/archive/refs/heads/develop.zip

In [ ]:
#loading the model
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1",
                                    use_auth_token="create_a_token_from_hugging_face")

In [ ]:
# apply the pipeline to an audio file
diarization = pipeline(audio_file,max_speakers=3)

In [ ]:
# dump the diarization output to disk using RTTM format
with open("Sys_genereated.rttm", "w") as rttm:
    diarization.write_rttm(rttm)

Creating a json file from the rttm file


In [ ]:
import json

# Define the path to the RTTM file
rttm_file = ""           ## insert path to system generated rttm file

# Define the path to the output JSON file
json_file = "/content/"+audio_file[:-4]+".json"

# Create an empty list to store the speaker turns
speaker_turns = []

# Open the RTTM file for reading
with open(rttm_file, "r") as f:
    # Iterate over each line in the file
    for line in f:
        # Split the line into fields
        fields = line.strip().split(" ")
        
        # Extract the relevant information
        speaker_id = fields[7]
        start_time = float(fields[3])
        end_time = start_time + float(fields[4])
        
        # Create a dictionary for the speaker turn
        speaker_turn = {
            "label": speaker_id,
            "start": start_time,
            "stop": end_time
        }
        
        # Append the speaker turn to the list
        speaker_turns.append(speaker_turn)

# Create a dictionary for the JSON object
json_data = {
    "text": str(speaker_turns)
}

# Save the JSON data to the output file
with open(json_file, "w") as f:
    json.dump(json_data, f)


Segmenting the audio file & Merging the same speaker files

In [ ]:
pip install pydub

In [ ]:
import json

# Define the path to the JSON file
json_file_path = json_file

# Open the JSON file for reading
with open(json_file_path, "r") as f:
    # Load the JSON data into a Python variable
    json_data = json.load(f)

In [ ]:
from pydub import AudioSegment


# Extract speech segments from JSON data
speech_segments = eval(json_data["text"])

# Load audio file
audio_file = AudioSegment.from_wav(audio_file)

# Create a dictionary to store segmented audio for each speaker ID
segmented_audio_dict = {}

# Iterate through speech segments
for segment in speech_segments:
    start_time = segment["start"] * 1000  # Convert to milliseconds
    end_time = segment["stop"] * 1000  # Convert to milliseconds
    speaker_id = segment["label"]

    # Segment audio file
    segmented_audio = audio_file[start_time:end_time]

    # If speaker ID already exists in the dictionary, concatenate the audio segment
    if speaker_id in segmented_audio_dict:
        segmented_audio_dict[speaker_id] += segmented_audio
    # If speaker ID is new, add the audio segment as a new entry in the dictionary
    else:
        segmented_audio_dict[speaker_id] = segmented_audio

# Merge audio segments from the same speaker ID
for speaker_id, segmented_audio in segmented_audio_dict.items():
    # Merge audio segments from the same speaker ID
    merged_audio = segmented_audio[0]
    for i in range(1, len(segmented_audio)):
        merged_audio += segmented_audio[i]

    # Update the dictionary with the merged audio
    segmented_audio_dict[speaker_id] = merged_audio

# Save merged audio for each speaker ID
for speaker_id, merged_audio in segmented_audio_dict.items():
    output_filename = f"merged_audio_{speaker_id}.wav"  # Generate output filename based on speaker ID
    merged_audio.export(output_filename, format="wav")  # Save merged audio as WAV file
    print(f"Merged audio for Speaker ID '{speaker_id}' saved as '{output_filename}'")

print("Segmentation and merging complete!")


#Evaluation
##Diarization error rate

* Here we used simpleder module for calculating DER.
* rttm_file refers to the system generated RTTM file.

In [ ]:
pip install simpleder

In [ ]:
import simpleder

# Define the path to the RTTM file
rttm_file = "" #path to system rttm file

# Define the path to the hypothesis RTTM file (same as the reference for this example)
hyp_file = "" #path to the grouth truth rttm file

# Open the RTTM file for reading
with open(rttm_file, "r") as f:
    # Create an empty list to store the reference speaker turns
    ref_turns = []

    # Iterate over each line in the file
    for line in f:
        # Split the line into fields
        fields = line.strip().split(" ")

        # Extract the relevant information
        speaker_id = fields[7]
        start_time = float(fields[3])
        end_time = start_time + float(fields[4])

        # Append the speaker turn to the list
        ref_turns.append((speaker_id, start_time, end_time))

# Open the hypothesis RTTM file for reading
with open(hyp_file, "r") as f:
    # Create an empty list to store the hypothesis speaker turns
    hyp_turns = []

    # Iterate over each line in the file
    for line in f:
        # Split the line into fields
        fields = line.strip().split(" ")

        # Extract the relevant information
        speaker_id = fields[7]
        start_time = float(fields[3])
        end_time = start_time + float(fields[4])

        # Append the speaker turn to the list
        hyp_turns.append((speaker_id, start_time, end_time))

# Compute the DER between the reference and hypothesis speaker turns
error = simpleder.DER(ref_turns, hyp_turns)

print("DER={:.3f}".format(error))